<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/ETL_C660.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
def get_col(df,name):
    return df.columns[df.columns.str.contains(name)].tolist()

In [39]:
path = '/content/drive/MyDrive/台塑輕油案子/data/c620/raw/c660/Aroma-2 C660_Split Factors Calculation for AI Training_Overall_Data.xlsx'

In [40]:
xl = pd.ExcelFile(path)

In [41]:
def clean_data(df):
    df = df.T.reset_index()
    idx = df.iloc[6:,1].values
    df = df.iloc[:,2:]
    prefix = pd.Series(df.iloc[1,:].values).fillna(method='ffill').values
    middle = pd.Series(df.iloc[2,:].values).fillna(method='ffill').values
    middle_2 = pd.Series(df.iloc[3,:].values).fillna(method='ffill').values
    suffix = pd.Series(df.iloc[4,:].values).fillna(method='ffill').values
    col = [p.strip() + '_' + m1.strip() + '_' + m2.strip() + '_' + s.strip() for p,m1,m2,s in zip(prefix,middle,middle_2,suffix)]
    df = df.iloc[6:,:]
    df.index = idx
    df.columns = col
    return df.astype('float')

In [42]:
train = pd.DataFrame()
for i in tqdm(xl.sheet_names):
    rows = clean_data(xl.parse(i))
    if len(train) == 0:
      train = rows
    else:
      train = train.append(rows)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [43]:
train.head()

,Combined Feed to Benzene Column C660_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Benzene Column C660_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Benzene Column C660_Detailed Composition_Hydrogen_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Methane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Propane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Butane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Pentane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Hexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Benzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Cyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Heptane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Water_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Toluene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Octane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_p-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_m-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_o-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Nonane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_Indane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Benzene Column C660_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_n-Pentylbenzene_wt%,...,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Methane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Ethane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Propane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_n-Butane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_n-Pentane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_n-Hexane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Benzene_Fract

In [45]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
# Missing values statistics
missing_values = missing_values_table(train)
missing_values.head(20)

Your selected dataframe has 418 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Sum_wt%,15552,100.0
Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Vent Gas_Vent Gas Split Factor_wt%,15552,100.0
Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Distillate_Dist Split Factor_Fraction,15552,100.0
Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Sidedraw_Sidedraw Split Factor_Fraction,15552,100.0
Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Btms Split Factor_Fraction,15552,100.0


In [47]:
print(train.shape)
train = train.dropna(axis=1)
train.shape

(15552, 418)


(15552, 413)

In [51]:
fr = get_col(train,'Mass Flow Rate')
train[fr]

,Combined Feed to Benzene Column C660_Flow Rate_Mass Flow Rate_ton/hr,Benzene Column C660 Operation_Feed Properties_Mass Flow Rate_ton/hr,Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Mass Flow Rate_ton/hr,Benzene Column C660 Operation_Distillate (Benzene Drag) Production Rate and Composition_Mass Flow Rate_ton/hr,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Mass Flow Rate_ton/hr,Benzene Column C660 Operation_Bottoms Production Rate and Composition_Mass Flow Rate_ton/hr
001-001,54.072098,54.072098,0.002363,1.961175,33.957754,18.150811
001-002,54.072086,54.072086,0.005511,1.656645,34.258637,18.151299
001-003,54.072090,54.072090,0.008100,1.444854,34.467496,18.151637
001-004,53.705863,53.705863,0.001373,2.036881,33.549375,18.118234
001-005,53.705867,53.705867,0.004676,1.694960,33.887500,18.118729
...,...,...,...,...,...,...
144-104,115.464344,115.464344,0.040084,0.211555,44.675133,70.537578
144-105,115.464359,115.464359,0.036053,0.299345,44.590965,70.537992
144-106,120.501742,120.501742,0.052597,0.161890,44.735066,75.552203
144-107,120.501742,120.501742,0.047360,0.254926,44.647266,75.552203


In [52]:
fr.remove(fr[1])
check_df = train[fr]
check_df.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_mbcheck.csv')

In [53]:
x41 = get_col(train,'Detailed Composition')[:-1]
case = get_col(train,'Specifications')[1:-1]

yRefluxRate = get_col(train, 'Reflux Rate')
yControl = get_col(train, 'Control')
yHeatDuty = get_col(train, 'Heat Duty_Mkcal')

vent_gas_x = get_col(train,'Operation_Vent Gas')[2:-1]
distillate_x = get_col(train,'Operation_Distillate')[2:-1]
sidedraw_x = get_col(train,'Operation_Sidedraw')[2:-1]
bottoms_x = get_col(train,'Operation_Bottoms')[2:-1]

vent_gas_sf = get_col(train,'C660 Vent Gas')
distillate_sf = get_col(train,'C660 Distillate')
sidedraw_sf = get_col(train,'C660 Sidedraw')
bottoms_sf = get_col(train,'C660 Bottoms')

In [63]:
import numpy as np
def sp2wt(x,s):
  a = 100*x*s
  b = np.diag(x@s.T).reshape(-1,1)
  b = np.clip(b,1e-8,np.inf)
  return a/b

In [65]:
for sf,wt in tqdm(zip([vent_gas_sf,distillate_sf,sidedraw_sf,bottoms_sf],[vent_gas_x,distillate_x,sidedraw_x,bottoms_x])):
  assert np.allclose(sp2wt(train[x41].values,train[sf].values),train[wt].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [66]:
col_names = {}
col_names['x41'] = x41
col_names['case'] = case

col_names['yRefluxRate'] = yRefluxRate
col_names['yControl'] = yControl
col_names['yHeatDuty'] = yHeatDuty

col_names['vent_gas_x'] = vent_gas_x
col_names['distillate_x'] = distillate_x
col_names['sidedraw_x'] = sidedraw_x
col_names['bottoms_x'] = bottoms_x

col_names['vent_gas_sf'] = vent_gas_sf
col_names['distillate_sf'] = distillate_sf
col_names['sidedraw_sf'] = sidedraw_sf
col_names['bottoms_sf'] = bottoms_sf

In [72]:
MFR = get_col(train, 'Mass Flow Rate')
MFR.remove(MFR[0])

VFR = get_col(train, 'Volume Flow Rate')
VFR.remove(VFR[0])

vMFR = train[MFR].values
vVFR = train[VFR].values
n = [ i.split('_')[1] for i in MFR]

col_names['density'] = ['Density_{}'.format(i) for i in n]
density = vMFR/vVFR
density = pd.DataFrame(density,columns=col_names['density'])
density

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition
0,0.878584,0.527970,0.871164,0.883697,0.870042
1,0.878584,0.532121,0.869987,0.883697,0.870040
2,0.878584,0.535616,0.868997,0.883697,0.870039
3,0.878581,0.526486,0.871840,0.883705,0.870041
4,0.878581,0.530887,0.870613,0.883705,0.870039
...,...,...,...,...,...
15547,0.875518,0.573158,0.852115,0.883697,0.870746
15548,0.875518,0.568720,0.855229,0.883705,0.870745
15549,0.875288,0.576937,0.849084,0.883689,0.870758
15550,0.875288,0.572252,0.852850,0.883697,0.870758


In [73]:
col_names.keys()

dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])

In [75]:
train = train.join(density)
print(train.shape)

(15552, 418)


In [77]:
for k,v in tqdm(col_names.items()):
  assert len(train[v]) != 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [78]:
train.head()

,Combined Feed to Benzene Column C660_Flow Rate_Volume Flow Rate_m3/hr,Combined Feed to Benzene Column C660_Flow Rate_Mass Flow Rate_ton/hr,Combined Feed to Benzene Column C660_Detailed Composition_Hydrogen_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Methane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Propane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Butane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Pentane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Hexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Benzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Cyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Heptane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Water_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Methylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Toluene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Octane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylcyclopentane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_Ethylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_p-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_m-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_o-Xylene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Nonane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_i-Propylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylcyclohexane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_n-Propylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-3-ethylbenzene_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-4-ethylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,3,5-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-2-ethylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2,4-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_tert-Butylcyclohexane_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2,3-Trimethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_Indane_wt%,Combined Feed to Benzene Column C660_Detailed Composition_1-Methyl-4-n-propylbenzene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,2-Diethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_5-Ethyl-m-xylene_wt%,"Combined Feed to Benzene Column C660_Detailed Composition_1,4-Diethylbenzene_wt%","Combined Feed to Benzene Column C660_Detailed Composition_1,2,3,5-Tetramethylbenzene_wt%",Combined Feed to Benzene Column C660_Detailed Composition_n-Pentylbenzene_wt%,...,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_n-Hexane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Benzene_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Cyclohexane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_n-Heptane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Water_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_Methylcyclohexane_Fraction,Benzene Column C660 Split Factors Calculation_Split Factor for Individual Component to Benzene Column C660 Bottoms_T

In [79]:
import joblib
train.to_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/{}_train.csv'.format('c660'))
joblib.dump(col_names,'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/{}_col_names.pkl'.format('c660'))
print('ok')

ok
